# Groq API - Adding Tensors

In this tutorial, we show how to do a simple addition between two float16 tensors.  This program manually allocates memory for the 2 input tensors and the result tensor.  We demonstrate how to use Groq API as well as NumPy to compare the methods and confirm the results. The tutorial will also compile the program and run on the GroqChip. 

By the end of this tutorial, you should feel comfortable with the following concepts:
* Time
* Components
* VXM pointwise add
* GroqView

It is expected that you have finished reading the Adding Tensors section of the Groq API Tutorial Guide prior to going through this tutorial. 

## Build a program and compile with Groq API

Begin by importing Groq API.

In [ ]:
import groq.api as g
from groq.runner import tsp

print("Python Packages imported successfully!")

We first declare the two tensors that we'll add together as `input_tensors`. This allocates memory for runtime input data and handles moving the data from the host memory to the GroqChip.
This example uses float16, however, the following data types: int8, int16, int32, float16, float32 are also supported. 

In [ ]:
t1_mt = g.input_tensor(shape=(2, 32), dtype=g.float16, name="tensor_1")
t2_mt = g.input_tensor(shape=(2, 32), dtype=g.float16, name="tensor_2")

`t1_mt` and `t2_mt` are declared as input tensors with shape `(2,32)` and type `float16`. By convention we append `_mt` to the tensor variable name to denote a memory tensor and `_st` for a streaming tensor. It is recommended to name all tensors. This will help with debug as well as viewing the tensor using GroqView. 

The following will create a component for the addition of the tensors. There are two stages of a component, as follows:

1. <b> Initialization </b>- components create resource requests, static tensors, and child components.
2. <b> Build</b> - function units are allocated (in this case the ALU for the add and the streams for the tensors) and time is assigned

In [ ]:
class Add_Component(g.Component):          # call the component class from Groq API
    def __init__(self):    # Stage 1: Initialization
        super().__init__()
    def build(self, t1_in, t2_in, time=0):         # Stage 2: Build
        add_result_st = t1_in.add(t2_in, time=time)             # Add t1 to t2, these values are passed to the component
        add_result_mt = add_result_st.write(name="add_result_mt")    # Write the result to memory and return the memory tensor
        return add_result_mt

Note that in the Add Component definition, we tell it to expect two input tensors (t1_in, t2_in). A good rule of thumb:
* Input tensors do not go in components, they're passed to components.
* We pass a start time to a component so that we can compose multiple components together and make sure they have the desired offset in time from each other. Another method is to use Buffered Resource Scopes (see Buffered Scopes Tutorial)

As well, we see that time is assigned on the add operation. The Groq API backend will work from the time specified on the add to schedule the required reads and writes, taking into account the travel time for the data. 

Next we can instantiate and call our component, providing the two tensors we declared earlier. 

In [ ]:
my_add_func = Add_Component()
add_result = my_add_func(t1_mt, t2_mt, time=50)     #pass both input tensors and time

The result of adding the two tensors will be `add_result`. Since we've assigned time = 50, starting at Cycle 50, the GroqChip will read the tensors from memory, a few cycles later, the VXM will perform an addition, and the last step will be to write it back to memory.

By this point, we have a program that adds two `float16` tensors. The next step is to compile this program, generating the binary code to be loaded onto the GroqChip.

In [ ]:
iop_file = g.compile(base_name="add_tensors", result_tensor=add_result)
print(iop_file)

`add_tensors.iop` contains the binary compiled program. This will be used to program the GroqChip with the desired functionality of our add tensor program. 

## GroqView
The GroqView can be used to view the instructions of your program in the GroqChip. Note: it is expected that you are familiar with the GroqView tool (See "GroqView User Guide") for this section of the tutorial. You may skip viewing the program in GroqView and move to the "Prepare Data for Program" section.

Using the following command, we can create a .json file that can be used to view the program in hardware. This will show:
* what instructions occur
* where on the chip they take place, as well as 
* when in time (cycles) each instruction occurs. 

In [ ]:
g.write_visualizer_data("add_tensors")

To launch the GroqView, uncomment and run the following command. Remember, you still need to create a tunnel to the server running GroqView to load in another window. 

<b>Note:</b> before proceeding to the next section, you'll want to stop this cell. 

In the GroqView tool, you should see the input tensors being read from memory, streamed to the VXM where they are added together and the result written back into memory. 

In [ ]:
#!groqview add_tensors/visdata.json

## Add Program to GroqChip

The next step is to use the create_tsp_runner function to load the program on the GroqChip. This will use the binary executable we compiled earlier to program the chip. 

In [ ]:
add_program = tsp.create_tsp_runner(iop_file)

At this point the binary program `add_tensors.iop` has been loaded on the GroqChip. We can now start sending data from the host to the program. 

## Prepare Data for Program

Using NumPy, we'll create two randomly generated tensors as inputs. We ensure that the shape of the tensors and the data type match what the GroqChip is programmed for: (2, 32) and float16.

In [ ]:
import numpy as np

shape = (2, 32)
t1_data = np.random.rand(2, 32).astype(np.float16)
t2_data = np.random.rand(2, 32).astype(np.float16)

Now that we have input data, let's send the tensors to the GroqChip and it will return the result of adding the two tensors together. The arguments to `add_program` are the input tensors for the loaded program. Here, `tensor_1` is the name of the input tensor previously allocated at the beginning, and `t1_data` is the tensor containing the input data.

In [ ]:
result = add_program(tensor_1=t1_data, tensor_2=t2_data)

Let's check that the results from the GroqChip are correct by comparing them with the result calculated via NumPy:

In [ ]:
numpy_result = t1_data + t2_data
np.array_equal(result['add_result_mt'], numpy_result)

## Back to Back Computations

The GroqChip is still programmed with the `add tensors` program so we can continue to provide input data and it will return the results of adding them together. Let's look at how we can perform calls to the same program repeatedly with different input tensors and check the results with NumPy.

In [ ]:
for i in range(3):
    print(f"Inference {i}")
    t1_data = np.random.rand(2, 32).astype(np.float16)
    t2_data = np.random.rand(2, 32).astype(np.float16)
    result = add_program(tensor_1=t1_data, tensor_2=t2_data)
    numpy_result = t1_data + t2_data
    np.array_equal(result['add_result_mt'], numpy_result)
    print(f"Validated {i} -->", np.array_equal(result['add_result_mt'], numpy_result))